**A jupyter notebook that demonstrates how to:**

1. Connect to Oracle database;
2. Run a query limiting sizes of results;
3. Create a csv file with results.


In [1]:
#installing cx_Oracle wich is not present at docker
!pip install cx_Oracle

    100% |████████████████████████████████| 737kB 316kB/s ta 0:00:01
You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals
from IPython.display import display,clear_output

import pandas as pd

import cx_Oracle
import os
import datetime

ip = 'database'
port = 1521
SID = 'ORCLCDB'
user = 'sys'
password = 'Oradoc_db1'

dsn_tns = cx_Oracle.makedsn(ip, port, SID)

connection = cx_Oracle.connect(user, 
                               password, 
                               dsn_tns, 
                               encoding = "UTF-8", 
                               nencoding = "UTF-8")

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "libclntsh.so: cannot open shared object file: No such file or directory". See https://oracle.github.io/odpi/doc/installation.html#linux for help

In [0]:
mges_por_eee = [2,2,3,3,3,1,2,3,2,2,3,3,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2]

In [0]:
for idx in range(1,31):
    query = """select to_char(INTIME, 'yyyy-mm-dd HH24:MI:SS') as INTIME_,
                      to_char(OUTTIME, 'yyyy-mm-dd HH24:MI:SS') as OUTTIME_,
                      MESSAGE, TAG
               FROM SYSTEM.ALARMS
               WHERE AREA = 'UTR_%02d' 
               ORDER BY INTIME"""%(idx)

    reader = pd.read_sql(query, connection, chunksize=50000)
    i = 0
    outfilename = r'csv_datasets/alarms_utr_%02d.csv' % (idx)
    outfilenamew = r'csv_datasets/alarms_utr_%02dw.csv' % (idx)    
    #df = pd.DataFrame()
    for chunk in reader:
      #query = """select * from alarms_utr14 where to_char(INTIME_, 'yyyy') = '%d'"""%(idx)
      #df = df.append(chunk)


      temp_regs = chunk
      if os.path.isfile(outfilename):
        temp_regs.to_csv(outfilename,encoding='utf-8', mode='a', header=False)
      else:
        temp_regs.to_csv(outfilename,encoding='utf-8')
    print(idx)

    fw = open(outfilenamew, 'w', encoding = "utf-8")
    last_row = None
    last_dh = None
    cur_values = []
    idxf = 0

    with open(outfilename, 'r', encoding = "utf-8") as fr:
        for line in fr:
            curdh = line.split(',',2)[1]
            curline = line.split(',',1)[1]
            if curdh != last_dh:
                cur_values = []

            if ((last_row == None) or (curline != last_row)) and (curline not in cur_values):
                if idxf == 0:
                    fw.write(',%s'%(curline))
                else:
                    fw.write('%d,%s'%(idxf - 1,curline))            
                idxf += 1
                cur_values.append(curline)
            last_row = curline
            last_dh = curdh
    fr.close()
    fw.close()

        

    #df = pd.read_csv(outfilename)

    #df.sort_values('INTIME_')
    #df.drop_duplicates(subset=None, keep='first', inplace=True)
    #df.sort_values(by=['INTIME_'])
    #df.drop_duplicates(subset=None, keep='first', inplace=True)
    #df.to_csv(outfilename,encoding='utf-8')

In [0]:
for utr in range(1,31):
    for mge in range(1,mges_por_eee[utr - 1] +1 ):
        query = """select to_char(e3timestamp, 'yyyy-mm-dd HH24:MI:SS') as timestamp, IA,IB,IC,URS,UST,URT,PA,FP 
                   from SYSTEM.INST_UTR_%02d_MGE%02d_%d""" % (utr, mge,utr)
    
        reader = pd.read_sql(query, connection, chunksize=50000)
        i = 0
        outfilename = r'csv_datasets/mge_utr_%02d_%d.csv' % (utr,mge)
        for chunk in reader:
            temp_regs = chunk
            if os.path.isfile(outfilename):
                temp_regs.to_csv(outfilename,encoding='utf-8', mode='a', header=False)
            else:
                temp_regs.to_csv(outfilename,encoding='utf-8')

In [0]:
for utr in range(1,31):
    query = """select to_char(e3timestamp, 'yyyy-mm-dd HH24:MI:SS') as timestamp, a.*
               from SYSTEM.TAB_INST_UTR_%02d a""" % (utr)

    reader = pd.read_sql(query, connection, chunksize=50000)
    i = 0
    outfilename = r'csv_datasets/tab_inst_utr_%02d.csv' % (utr)
    for chunk in reader:
        temp_regs = chunk
        if os.path.isfile(outfilename):
            temp_regs.to_csv(outfilename,encoding='utf-8', mode='a', header=False)
        else:
            temp_regs.to_csv(outfilename,encoding='utf-8')

In [0]:
query = """select to_char(e3timestamp, 'yyyy-mm-dd HH24:MI:SS') as e3timestamp,
                  to_char(eventtime, 'yyyy-mm-dd HH24:MI:SS') as eventtime, EVENTMESSAGE, EVENTCOMMENT
           from SYSTEM.TAB_EVENTS"""

reader = pd.read_sql(query, connection, chunksize=50000)
i = 0
outfilename = r'csv_datasets/tab_events.csv'
for chunk in reader:
    temp_regs = chunk
    if os.path.isfile(outfilename):
        temp_regs.to_csv(outfilename,encoding='utf-8', mode='a', header=False)
    else:
        temp_regs.to_csv(outfilename,encoding='utf-8')

In [0]:
for utr in range(1,31):
    query = """select to_char(e3timestamp, 'yyyy-mm-dd HH24:MI:SS') as timestamp, a.*
               from SYSTEM.TAB_TOTDIA_UTR_%02d a""" % (utr)

    reader = pd.read_sql(query, connection, chunksize=50000)
    i = 0
    outfilename = r'csv_datasets/tab_totdia_utr_%02d.csv' % (utr)
    for chunk in reader:
        temp_regs = chunk
        if os.path.isfile(outfilename):
            temp_regs.to_csv(outfilename,encoding='utf-8', mode='a', header=False)
        else:
            temp_regs.to_csv(outfilename,encoding='utf-8')

In [0]:
query = """select to_char(e3timestamp, 'yyyy-mm-dd HH24:MI:SS') as e3timestamp,
                  to_char(evento, 'yyyy-mm-dd HH24:MI:SS') as eventtime, a.*
           from SYSTEM.HIST_OCORRENCIAS a"""

reader = pd.read_sql(query, connection, chunksize=50000)
i = 0
outfilename = r'csv_datasets/hist_ocorrencias.csv'
for chunk in reader:
    temp_regs = chunk
    if os.path.isfile(outfilename):
        temp_regs.to_csv(outfilename,encoding='utf-8', mode='a', header=False)
    else:
        temp_regs.to_csv(outfilename,encoding='utf-8')